In [ ]:
#Importamos las liberías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import seaborn as sns
%matplotlib inline

from pandas import read_csv
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
#import pydotplus
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")

## PREPARACIÓN DE DATOS

In [ ]:
#Cargamos el csv
df=pd.read_csv("obesidad_ds.csv",sep=",")
df

In [ ]:
#Cambiar los nombres de las columnas
nuevos_nombres_col = ['genero', 'edad', 'altura', 'peso', 'hist_fam', 'alta_cal_frec', 'verduras', 'num_comidas', 'entrehoras', 'fumar', 'agua_dia', 'monit_cal', 'act_fis', 'uso_tecn', 'alcohol', 'transporte', 'diagnostico']
df.columns = nuevos_nombres_col


In [ ]:
#Ver los valores de cada columna
for i in df.columns:
    print(df[i].unique())

In [ ]:
#Cambiar valores a numéricos
nom_genero={'Female': 0, 'Male': 1}
df['genero'] = df['genero'].replace(nom_genero)

nom_hist_fam={'no': 0, 'yes': 1}
df['hist_fam'] = df['hist_fam'].replace(nom_hist_fam)

nom_alta_cal_frec={'no': 0, 'yes': 1}
df['alta_cal_frec'] = df['alta_cal_frec'].replace(nom_alta_cal_frec)

nom_entrehoras={'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
df['entrehoras'] = df['entrehoras'].replace(nom_entrehoras)

nom_fumar={'no': 0, 'yes': 1}
df['fumar'] = df['fumar'].replace(nom_fumar)

nom_monit_cal={'no': 0, 'yes': 1}
df['monit_cal'] = df['monit_cal'].replace(nom_monit_cal)

nom_alcohol={'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
df['alcohol'] = df['alcohol'].replace(nom_alcohol)

nom_transporte={'Public_Transportation': 1, 'Walking': 2, 'Automobile': 3, 'Motorbike': 4, 'Bike': 5}
df['transporte'] = df['transporte'].replace(nom_transporte)

nom_diagnostico = {'Normal_Weight': 'normal', 'Insufficient_Weight': 'insuficiente', 'Overweight_Level_I': 'sobrepeso1', 'Overweight_Level_II':'sobrepeso2', 'Obesity_Type_III':'obesidad3','Obesity_Type_II':'obesidad2', 'Obesity_Type_I':'obesidad1'}
df['diagnostico'] = df['diagnostico'].replace(nom_diagnostico)


In [ ]:
#Vemos los valores que tiene cada columna y cuantas veces aparece cada uno de ellos
for i in df.columns:
    print(i,df[i].value_counts())

In [ ]:
#Información del dataset
df.info()

In [ ]:
#Cambiar tipo de valores
#df['genero'] = df['genero'].astype(int)
#df

In [ ]:
frecuencia_clase = df['diagnostico'].value_counts()

# Define el orden deseado de los diagnósticos
orden_diagnosticos = ['insuficiente', 'normal', 'sobrepeso1', 'sobrepeso2', 'obesidad1', 'obesidad2', 'obesidad3']

# Ordena los valores y diagnósticos en función del orden deseado
frecuencia_clase = frecuencia_clase.loc[orden_diagnosticos]

plt.barh(frecuencia_clase.index, frecuencia_clase.values, color="lightgreen")
plt.xlabel('Frecuencia')
plt.ylabel('Diagnóstico')
plt.show()

In [ ]:
nom_diagnostico={'insuficiente': 1, 'normal': 2, 'sobrepeso1': 3, 'sobrepeso2': 4, 'obesidad1': 5, 'obesidad2': 6, 'obesidad3': 7}
df['diagnostico'] = df['diagnostico'].replace(nom_diagnostico)
df

In [ ]:
#Guardamos el dataset como csv
df.to_csv("obesidad_py.csv", index=False)

# PREPROCESAMIENTO

In [ ]:
# Dividimos entrada y salida
x=df.iloc[:,0:16]
y=df[["diagnostico"]]
# Dividimos en datos de entrenamiento y de validación
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True)

# Tamaños
x.shape, y.shape

x_train.shape, y_train.shape

x_test.shape, y_test.shape

# Calculamos el máximo y mínimo
maxs = np.max(x_train, axis=0)
mins = np.min(x_train, axis=0)

ranges = maxs - mins

# Normalizamos las variables
x_train = (x_train - mins) / ranges
x_test = (x_test - mins) / ranges

# MODELOS

## Árbol de decisión

In [ ]:
# modelo genérico
DT = DecisionTreeClassifier()
# entrenamos el modelo
DT.fit(x_train, y_train)
# hacer predicciones
pred_DT = DT.predict(x_test)
# informe
print(classification_report(y_test,pred_DT))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    DT, x_train, y_train, cv=kfold, scoring="accuracy"
)
DT_prec=score.mean()
print("cross_value: ", DT_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_DT)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

In [ ]:
# ver opciones con otros parametros
DT_combinaciones={}
# parametros
splitter_ = ["best", "random"]
criterion_ = ["gini", "entropy"]
max_features_ = [None,"sqrt", "log2"]
porc_ = [0.2, 0.3]
# combinaciones
for c in criterion_:
    for m in max_features_:
        for s in splitter_:
            for p in [1, 2]:
                vector_parametros=[c,m,s,porc_[p-1]]
                # dividimos las instancias
                x_train, x_test, y_train, y_test = train_test_split(
                    x, y, test_size=porc_[p - 1], random_state=42, shuffle=True
                )
                # modelo
                DT = DecisionTreeClassifier(criterion=c, max_features=m, splitter=s)
                # entrenamos el modelo
                DT.fit(x_train, y_train)
                # hacer predicciones
                predicciones_DT = DT.predict(x_test)
                # calculamos la exactitud del modelo
                acc_1 = accuracy_score(y_test, predicciones_DT)
                # cross value
                kfold = StratifiedKFold(10)
                score = cross_val_score(DT, x_train, y_train, cv=kfold, scoring="accuracy")
                print(
                    c,
                    ",",
                    m,
                    "y",
                    s,
                    "(",
                    porc_[p - 1],
                    ")",
                    "-> accuracy_score: ",
                    acc_1,
                    "//",
                    "cross_value: ",
                    score.mean()
                )
                # añadimos al diccionario esta combinación
                DT_combinaciones[score.mean()]=vector_parametros
                
# calculamos la precisión máxima
DT_max_prec = max(DT_combinaciones)
# miramos con que parámetros se consigue esta precisión
DT_mejor_vector_parametros = DT_combinaciones[DT_max_prec]
print("mejores resultados con los parametros -> ",DT_mejor_vector_parametros)

In [ ]:
# mejor modelo (utilizando el vector conseguido anteriormente)
DT_mejor = DecisionTreeClassifier(criterion=DT_mejor_vector_parametros[0], max_features=DT_mejor_vector_parametros[1], splitter=DT_mejor_vector_parametros[2])
# dividimos las instancias 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=DT_mejor_vector_parametros[3], random_state=42, shuffle=True)
# entrenamos el modelo
DT_mejor.fit(x_train, y_train)
# hacer predicciones
pred_DT_mejor = DT_mejor.predict(x_test)
# informe
print(classification_report(y_test,pred_DT_mejor))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    DT_mejor, x_train, y_train, cv=kfold, scoring="accuracy"
)
DT_mejor_prec=score.mean()
print("cross_value: ", DT_mejor_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_DT_mejor)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

## Random forest

In [ ]:
# modelo genérico
y_ravel= np.ravel(y)
RF = RandomForestClassifier()
# entrenamos el modelo
y_train=y_train.values.ravel()
RF.fit(x_train, y_train)
# hacer predicciones
pred_RF = RF.predict(x_test)
# informe
print(classification_report(y_test,pred_RF,zero_division=0))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    RF, x_train, y_train, cv=kfold, scoring="accuracy"
)
RF_prec=score.mean()
print("cross_value: ", RF_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_RF)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

In [ ]:
# ver opciones con otros parámetros
RF_combinaciones={}
# parametros
criterion_ = ["gini", "entropy"]
max_features_ = [None,"sqrt", "log2"]
porc_ = [0.2, 0.3]
# combinaciones
for c in criterion_:
    for m in max_features_:
        for p in [1, 2]:
            vector_parametros=[c,m,porc_[p-1]]
            # dividimos las instancias
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=porc_[p - 1], random_state=42, shuffle=True)
            y_train=y_train.values.ravel()
            # modelo
            RF = RandomForestClassifier(criterion=c, max_features=m)
            # entrenamos el modelo
            RF.fit(x_train, y_train)
            # hacer predicciones
            predicciones_RF = RF.predict(x_test)
            # calculamos la exactitud del modelo
            acc_1 = accuracy_score(y_test, predicciones_RF)
            # cross value
            kfold = StratifiedKFold(10)
            score = cross_val_score(RF, x_train, y_train, cv=kfold, scoring="accuracy")
            print(
                c,
                "y",
                m,
                "(",
                porc_[p - 1],
                ")",
                "-> accuracy_score: ",
                acc_1,
                "//",
                "cross_value: ",
                score.mean()
            )
            # añadimos al diccionario esta combinación
            RF_combinaciones[score.mean()]=vector_parametros
                
# calculamos la precisión máxima
RF_max_prec = max(RF_combinaciones)
# miramos con que parámetros se consigue esta precisión
RF_mejor_vector_parametros = RF_combinaciones[RF_max_prec]
print("mejores resultados con los parametros -> ",RF_mejor_vector_parametros)

In [ ]:
# mejor modelo (utilizando el vector conseguido anteriormente)
RF_mejor = RandomForestClassifier(criterion=RF_mejor_vector_parametros[0], max_features=RF_mejor_vector_parametros[1])
# dividimos las instancias 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=RF_mejor_vector_parametros[2], random_state=42, shuffle=True)
y_train=y_train.values.ravel()
# entrenamos el modelo
RF_mejor.fit(x_train, y_train)
# hacer predicciones
pred_RF_mejor = RF_mejor.predict(x_test)
# informe
print(classification_report(y_test,pred_RF_mejor))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    RF_mejor, x_train, y_train, cv=kfold, scoring="accuracy"
)
RF_mejor_prec=score.mean()
print("cross_value: ", RF_mejor_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_RF_mejor)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

## Redes neuronales 

In [ ]:
optim=["adam","SGD","RMSprop","adagrad","adamax"]
RN_combinaciones={}

# Separa los atributos y las etiquetas de diagnóstico
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values -1

# Divide los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Escala los atributos para normalizarlos
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# ESTRUCTURA 1
RNmodel = Sequential()

# Agrega capas densas (totalmente conectadas) al modelo
RNmodel.add(Dense(64, activation='relu', input_shape=(16,)))  # Capa oculta 1
RNmodel.add(Dense(64, activation='relu'))  # Capa oculta 2
RNmodel.add(Dense(7, activation='softmax'))  # Capa de salida (7 niveles de obesidad)

for i in optim:
    # Compila el modelo
    RNmodel.compile(optimizer=i, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Entrena el modelo
    RNmodel.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

    y_pred_prob = RNmodel.predict(x_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    print(y_pred)
    # Calcular precisión
    precision = accuracy_score(y_test, y_pred)
    print("Precisión:", precision)

    # Calcular matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    ConfusionMatrixDisplay(matriz_confusion).plot()
    plt.show()
    
    RN_combinaciones[precision]=[1,i]
    
# ESTRUCTURA 2
RNmodel = Sequential()

# Agrega capas densas (totalmente conectadas) al modelo
RNmodel.add(Dense(128, activation='relu', input_shape=(16,)))  # Capa oculta 1
RNmodel.add(Dense(64, activation='relu'))  # Capa oculta 2
RNmodel.add(Dense(32, activation='relu'))  # Capa oculta 3
RNmodel.add(Dense(7, activation='softmax'))  # Capa de salida (7 niveles de obesidad)

for i in optim:
    # Compila el modelo
    RNmodel.compile(optimizer=i, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Entrena el modelo
    RNmodel.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

    y_pred_prob = RNmodel.predict(x_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    
    # Calcular precisión
    precision = accuracy_score(y_test, y_pred)
    print("Precisión:", precision)

    # Calcular matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    ConfusionMatrixDisplay(matriz_confusion).plot()
    plt.show()
    
    RN_combinaciones[precision] = [2, i]
    
# ESTRUCTURA 3
RNmodel = Sequential()

# Agrega capas densas (totalmente conectadas) al modelo
RNmodel.add(Dense(64, activation='relu', input_shape=(16,)))  # Capa oculta 1
RNmodel.add(Dense(128, activation='relu'))  # Capa oculta 2
RNmodel.add(Dense(64, activation='relu'))  # Capa oculta 3
RNmodel.add(Dense(32, activation='relu'))  # Capa oculta 4
RNmodel.add(Dense(7, activation='softmax'))  # Capa de salida (7 niveles de obesidad)

for i in optim:
    # Compila el modelo
    RNmodel.compile(optimizer=i, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Entrena el modelo
    RNmodel.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

    y_pred_prob = RNmodel.predict(x_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    
    # Calcular precisión
    precision = accuracy_score(y_test, y_pred)
    print("Precisión:", precision)

    # Calcular matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    ConfusionMatrixDisplay(matriz_confusion).plot()
    plt.show()
    
    RN_combinaciones[precision] = [3, i]
    
print(RN_combinaciones)
# calculamos la precisión máxima
RN_max_prec = max(RN_combinaciones)
# miramos con que parámetros se consigue esta precisión
RN_mejor_vector_parametros = RN_combinaciones[RN_max_prec]
print("mejores resultados con los parametros -> ",RN_mejor_vector_parametros)

In [ ]:
# Mejor modelo

RNmodel_mejor = Sequential()

if RN_mejor_vector_parametros[0]==1:
    RNmodel_mejor.add(Dense(64, activation='relu', input_shape=(16,)))  # Capa oculta 1
    RNmodel_mejor.add(Dense(64, activation='relu'))  # Capa oculta 2
    RNmodel_mejor.add(Dense(7, activation='softmax'))  # Capa de salida (7 niveles de obesidad)
elif RN_mejor_vector_parametros[0]==2:
    RNmodel_mejor.add(Dense(128, activation='relu', input_shape=(16,)))  # Capa oculta 1
    RNmodel_mejor.add(Dense(64, activation='relu'))  # Capa oculta 2
    RNmodel_mejor.add(Dense(32, activation='relu'))  # Capa oculta 3
    RNmodel_mejor.add(Dense(7, activation='softmax'))  # Capa de salida (7 niveles de obesidad)
elif RN_mejor_vector_parametros[0]==3:
    RNmodel_mejor.add(Dense(64, activation='relu', input_shape=(16,)))  # Capa oculta 1
    RNmodel_mejor.add(Dense(128, activation='relu'))  # Capa oculta 2
    RNmodel_mejor.add(Dense(64, activation='relu'))  # Capa oculta 3
    RNmodel_mejor.add(Dense(32, activation='relu'))  # Capa oculta 4
    RNmodel_mejor.add(Dense(7, activation='softmax'))  # Capa de salida (7 niveles de obesidad)
# Compila el modelo
RNmodel_mejor.compile(optimizer=RN_mejor_vector_parametros[1], loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrena el modelo
RNmodel_mejor.fit(x_train, y_train, epochs=40, batch_size=32, validation_data=(x_test, y_test))

y_pred_prob = RNmodel_mejor.predict(x_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calcular precisión
RN_mejor_prec = accuracy_score(y_test, y_pred)
print("Precisión:", RN_mejor_prec)

# Calcular matriz de confusión
matriz_confusion = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
ConfusionMatrixDisplay(matriz_confusion).plot()
plt.show()

## KNN

In [ ]:
avg_score=[]
max_score=0
best_k=0
for k in range(3,30):
    knn_model=KNeighborsClassifier(n_jobs=-1,n_neighbors=k)
    score=cross_val_score(knn_model,x_train,y_train,cv=5,scoring='accuracy')
    if score.mean()>max_score:
        max_score=score.mean()
        best_k=k
    avg_score.append(score.mean())

print([best_k,max(avg_score)])

max_index = avg_score.index(max(avg_score))
max_value = max(avg_score)

plt.figure(figsize=(15, 8))
plt.plot(range(3, 30), avg_score, color='lightgreen', linewidth=2)  
plt.scatter(max_index+2, max_value, color='green', marker='o', s=200)  
plt.xlabel("k neighbors")
plt.ylabel("accuracy")

plt.show()

In [ ]:
# modelo
KNN=KNeighborsClassifier(n_neighbors=best_k)
# entrenamos el modelo
KNN.fit(x_train,y_train)
# hacer predicciones
pred_KNN = KNN.predict(x_test)
# informe
print(classification_report(y_test,pred_KNN,zero_division=0))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    KNN, x_train, y_train, cv=kfold, scoring="accuracy"
)
KNN_prec=score.mean()
print("cross_value: ", KNN_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_KNN)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

# SVG

In [ ]:
# modelo
modelsvc = SVC()
# entrenamos el modelo
modelsvc.fit(x_train, y_train)
# hacer predicciones
pred_SVC = modelsvc.predict(x_test)
# informe
print(classification_report(y_test,pred_SVC,zero_division=0))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    modelsvc, x_train, y_train, cv=kfold, scoring="accuracy"
)
SVC_prec=score.mean()
print("cross_value: ", SVC_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_SVC)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()


In [ ]:
# ver opciones con otros parámetros
svc_combinaciones={}
# parametros
c_ = [1, 20, 100, 200]
kernel_ = ["linear", "poly", "rbf", "sigmoid"]
porc_ = [0.2, 0.3]
# combinaciones
for c in c_:
    for k in kernel_:
        for p in [1, 2]:
            vector_parametros=[c,k,porc_[p-1]]
            # dividimos las instancias
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=porc_[p - 1], random_state=42, shuffle=True)
            # modelo
            modelsvc = SVC(C=c, kernel=k)
            # entrenamos el modelo
            modelsvc.fit(x_train, y_train)
            # hacer predicciones
            predicciones_SVC = modelsvc.predict(x_test)
            # calculamos la exactitud del modelo
            acc_1 = accuracy_score(y_test, predicciones_SVC)
            # cross value
            kfold = StratifiedKFold(10)
            score = cross_val_score(modelsvc, x_train, y_train, cv=kfold, scoring="accuracy")
            print(
                c,
                "y",
                k,
                "(",
                porc_[p - 1],
                ")",
                "-> accuracy_score: ",
                acc_1,
                "//",
                "cross_value: ",
                score.mean()
            )
            # añadimos al diccionario esta combinación
            svc_combinaciones[score.mean()]=vector_parametros
                
# calculamos la precisión máxima
svc_max_prec = max(svc_combinaciones)
# miramos con que parámetros se consigue esta precisión
svc_mejor_vector_parametros = svc_combinaciones[svc_max_prec]
print("mejores resultados con los parametros -> ",svc_mejor_vector_parametros) 

In [ ]:
# mejor modelo (utilizando el vector conseguido anteriormente)
svc_mejor = SVC(C=svc_mejor_vector_parametros[0], kernel=svc_mejor_vector_parametros[1])
# dividimos las instancias 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=svc_mejor_vector_parametros[2], random_state=42, shuffle=True)
# entrenamos el modelo
svc_mejor.fit(x_train, y_train)
# hacer predicciones
pred_svc_mejor = svc_mejor.predict(x_test)
# informe
print(classification_report(y_test,pred_svc_mejor,zero_division=1))
# cross value
kfold = StratifiedKFold(10)
score = cross_val_score(
    svc_mejor, x_train, y_train, cv=kfold, scoring="accuracy"
)
svc_mejor_prec=score.mean()
print("cross_value: ", svc_mejor_prec)
# matriz de confusión
mat_conf = confusion_matrix(y_test, pred_svc_mejor)
ConfusionMatrixDisplay(mat_conf).plot()
plt.show()

# DIAGNOSIS

In [ ]:
nombre_v=str(input("Ahora introduciras los datos médicos del paciente con nombre o identificador: "))

genero_v=int(input("genero (0=mujer)(1=hombre): "))
edad_v=float(input("edad en años: "))
altura_v=float(input("altura en metros: "))
peso_v=float(input("peso en kg: "))
hist_fam_v=int(input("¿Tiene antecedentes familiares? (0=no)(1=si): "))
alta_cal_frec_v=int(input("¿Consume comida altamente calórica? (0=no)(1=si): "))
verduras_v=float(input("Cantidad de verduras semanales: "))
num_comidas_v=float(input("Cantidad de comidas diarias: "))
entrehoras_v=int(input("¿Come entre horas? (0=nunca)(1=a veces)(2=frecuentemente)(3=siempre): "))
fumar_v=int(input("¿Fuma? (0=no)(1=si): "))
agua_dia_v=float(input("Litros de agua diarios: "))
monit_cal_v=int(input("¿Monitoriza las calorías que consume? (0=no)(1=si): "))
act_fis_v=float(input("Días semanales de actividad física: "))
uso_tecn_v=float(input("Horas diarias de uso de la tecnología: "))
alcohol_v=int(input("¿Toma bebidas alcohólicas? (0=nunca)(1=a veces)(2=frecuentemente)(3=siempre): "))
transporte_v=int(input("¿Transporte habitual?(1=transporte público)(2=a pie)(3=automóvil)(4=motocicleta)(5=bicicleta): "))

In [ ]:
pac=[genero_v, edad_v, altura_v, peso_v, hist_fam_v, alta_cal_frec_v, verduras_v, num_comidas_v, entrehoras_v, fumar_v, agua_dia_v, monit_cal_v, act_fis_v, uso_tecn_v, alcohol_v, transporte_v]
#nombre_v="oijan"
#pac=[0,26,1.622,110.79,1,1,3,3,1,0,2.7,0,0,0.29,1,1]

# Crear los DataFrames que necesitaremos
df_diagnostico = pd.DataFrame(columns=['Método', 'Características', 'Precisión', 'Diagnóstico', 'Diagnóstico_nombre'])

df_datos=pd.DataFrame({'medida':['género', 'edad', 'altura', 'peso', 'hist_fam', 'alta_cal_frec', 'verduras','num_comidas','entrehoras','fumar','agua_dia','monit_cal','act_fis','uso_tecn','alcohol','transporte'],
                     'valor':pac})

df_nombre=pd.DataFrame({'nombre':[nombre_v]})

# Predecir diagnóstico con diferentes métodos
DT_predict=int(DT_mejor.predict([pac]))
RF_predict=int(RF_mejor.predict([pac]))
pac_scaled = scaler.transform([pac])  # Escala el vector pac
y_pred_prob = RNmodel_mejor.predict(pac_scaled)
RN_predict = np.argmax(y_pred_prob, axis=1)
KNN_predict=int(KNN.predict([pac]))
svc_predict=int(svc_mejor.predict([pac]))

# Añadir registros al DataFrame
def diag_nom(diagnostico):
    if diagnostico==0:
        return "insuficiente"
    elif diagnostico==1:
        return "normal"
    elif diagnostico==2:
        return "sobrepeso 1"
    elif diagnostico==3:
        return "sobrepeso 2"
    elif diagnostico==4:
        return "obesidad 1"
    elif diagnostico==5:
        return "obesidad 2"
    elif diagnostico==6:
        return "obesidad 3"
    
df_diagnostico = df_diagnostico.append({'Método': 'Árbol de decisión', 'Características': DT_mejor_vector_parametros, 'Precisión': DT_mejor_prec, 'Diagnóstico': DT_predict, 'Diagnóstico_nombre': diag_nom(DT_predict)}, ignore_index=True)
df_diagnostico = df_diagnostico.append({'Método': 'Random Forest', 'Características': RF_mejor_vector_parametros, 'Precisión': RF_mejor_prec, 'Diagnóstico': RF_predict, 'Diagnóstico_nombre': diag_nom(RF_predict)}, ignore_index=True)
df_diagnostico = df_diagnostico.append({'Método': 'Redes neuronales', 'Características': RN_mejor_vector_parametros, 'Precisión': RN_mejor_prec, 'Diagnóstico': RN_predict[0]+1, 'Diagnóstico_nombre': diag_nom(RN_predict)}, ignore_index=True)
df_diagnostico = df_diagnostico.append({'Método': 'KNN', 'Características': 'K='+ str(best_k), 'Precisión': KNN_prec, 'Diagnóstico': KNN_predict, 'Diagnóstico_nombre': diag_nom(KNN_predict)}, ignore_index=True)
df_diagnostico = df_diagnostico.append({'Método': 'SVC', 'Características': svc_mejor_vector_parametros, 'Precisión': svc_mejor_prec, 'Diagnóstico': svc_predict, 'Diagnóstico_nombre': diag_nom(svc_predict)}, ignore_index=True)

df_diagnostico


In [ ]:
#Guardamos los dataset como csv
df_datos.to_csv("datospac_obesidad_py.csv", index=False)
df_diagnostico.to_csv("diagnostico_obesidad_py.csv", index=False)
df_nombre.to_csv("nombrepac_obesidad_py.csv", index=False)
